## Dataframe and Spark SQL

There are two ways of create Spark Dataframes
1. Create from an existing RDD
2. Create from other data sources

Creating DataFrames from Existing RDD:
1. Infer schema by Reflection
      - Convert RDD containing case classes.
      - Use when schema is known.
2. Construct schema programmatically
      - Schema is dynamic
      - Number of fields in case class is more than 22 fields.

#### Inferring the Schema Using Reflection

In [4]:
from pyspark.sql import SQLContext, Row 
sqlContext = SQLContext(sc) 

# Load a text file and convert each line to a Row.
lines = sc.textFile("file:/datasets/crime_incidents.csv") 
parts = lines.map(lambda l: l.split(",")) 
crime = parts.map(lambda p: Row(IncidntNum=p[0], Category =p[1], Descript=p[2], day=p[3],date=p[4],time=p[5],district=p[6],resolution=p[7],address=p[8],X=p[9],Y=p[10],location=p[11]))

# Infer the schema, and register the DataFrame as a table.
schemaCrime = sqlContext.createDataFrame(crime) 
schemaCrime.registerTempTable("incidents")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = sqlContext.sql("SELECT * from incidents") 
teenagers.show()


+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------+--------------+--------------------+
|            Category|            Descript|IncidntNum|                   X|                   Y|             address|                date|                 day|  district|          location|    resolution|                time|
+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------+--------------+--------------------+
|            Category|            Descript|IncidntNum|                   X|                   Y|             Address|                Date|           DayOfWeek|PdDistrict|          Location|    Resolution|                Time|
|             ASSAULT|             BATTERY| 050436712|   -122.435002864271|    37.7608878061245|

In [8]:
#teenNames = teenagers.map(lambda p: "Name: " + p.name) 
#for teenName in teenNames.collect(): 
    #print(teenName)

#### Programmatically Specifying the Schema

A DataFrame can be created programmatically with three steps.
- Create an RDD of tuples or lists from the original RDD;
- Create the schema represented by a StructType matching the structure of tuples or lists in the RDD created in the step 1.
- Apply the schema to the RDD via createDataFrame method provided by SQLContext

In [10]:
# Load a text file and convert each line to a tuple.
lines = sc.textFile("file:/datasets/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: (p[0], p[1].strip()))

# The schema is encoded in a string.
schemaString = "name age" 
from pyspark.sql.types import * 
fields = [StructField(field_name, StringType(), True) 
for field_name in schemaString.split()] 
schema = StructType(fields)

# Apply the schema to the RDD and dataframe is created
schemaPeople = sqlContext.createDataFrame(people, schema)

# Print the schema in a tree format
schemaPeople.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)



In [11]:
# Print the rows of the dataframe
schemaPeople.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [12]:
# Register the DataFrame as a table.
schemaPeople.registerTempTable("people")

In [13]:
# SQL can be run over DataFrames that have been registered as a table.
results = sqlContext.sql("SELECT name FROM people") 
results.show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

